In [32]:
import dendropy
from dendropy.simulate import treesim
from dendropy.calculate import treecompare
import random

In [33]:
def print_generation(n):
    familly['F{}L_filtered'.format(n)].print_plot()
    print(familly['F{}L_mutation'.format(n)])

    familly['F{}R_filtered'.format(n)].print_plot()
    print(familly['F{}R_mutation'.format(n)])
    print(familly['F{}_URF'.format(n)])

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def swaaap(fam):
    swapy = []
    for n in fam.postorder_internal_node_iter():
    
        if (random.random() > 0.9 and n._parent_node and n._child_nodes ):
            swapy.append(n)
            
    if len(swapy) >= 2 :
    
        if len(swapy)%2 == 1:
            swapy.pop()
                
    
        for pair in chunks(swapy, 2):
            
            s0 = pair[0].extract_subtree()
            p0 = pair[0]._parent_node

            s1 = pair[1].extract_subtree()
            p1 = pair[1]._parent_node

            p0.add_child(s1)
            p1.add_child(s0)
            
            p0.remove_child(pair[0])
            p1.remove_child(pair[1])
            
    
    for n in fam.leaf_node_iter():
        if (n.taxon == None):
            n._parent_node.remove_child(n)
            

In [35]:
forest = []
size_forest = 200
n_generations = 4

for e in range(size_forest):
    
    to_store = {}
    
        
    familly = {}
    familly['tree_size'] = e+4
    to_store['tree_size'] = familly['tree_size']
    familly['tns'] = dendropy.TaxonNamespace(['T_{}'.format(i) for i in range(1,int(e * 1.5))])

    # ROOT PAIR
    familly['ROOT'] = treesim.birth_death_tree(1, 0.1, taxon_namespace=familly['tns'], num_extant_tips=familly['tree_size'])
    
    # RANDOM FILTER LEAF
    familly['L'] = dendropy.Tree(familly['ROOT'])
    familly['L'].filter_leaf_nodes(lambda n: random.random() < 0.8, recursive=False, update_bipartitions=True, suppress_unifurcations=True)
    for e in familly['L'].postorder_edge_iter():
        if random.random() > 0.7:
            e.length = e.length *1.5
        
    familly['R'] = dendropy.Tree(familly['ROOT'])
    familly['R'].filter_leaf_nodes(lambda n: random.random() < 0.8, recursive=False, update_bipartitions=True, suppress_unifurcations=True)
    for e in familly['R'].postorder_edge_iter():
        if random.random() > 0.7:
            e.length = e.length *1.5

    # RANDOM SWAP NODE
    if familly['tree_size'] > 30:
        swaaap(familly['R'])
        swaaap(familly['L'])
    
    to_store['L'] = familly['L'].as_string(schema='newick')
    to_store['R'] = familly['R'].as_string(schema='newick')
    
    familly['common'] = set(n.taxon for n in familly['L'].leaf_node_iter()) & set(n.taxon for n in familly['R'].leaf_node_iter())
    familly['common'].discard(None)
    familly['common'].discard('')
    
    familly['L_filter'] = familly['L'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    familly['R_filter'] = familly['R'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    
    # CLADE
    familly['R_filter'].is_rooted = True
    familly['L_filter'].is_rooted = True
    to_store['root_clade'] = treecompare.symmetric_difference( familly['L_filter'], familly['R_filter'])

    
    to_store['L_filter'] = familly['L_filter'].as_string(schema='newick')
    to_store['R_filter'] = familly['R_filter'].as_string(schema='newick')
    
    
   
    # RF + EUC
    familly['R_filter'].is_rooted = False
    familly['L_filter'].is_rooted = False
        
    to_store['root_URF'] = treecompare.symmetric_difference( familly['L_filter'], familly['R_filter'])
    to_store['root_WRF'] = treecompare.weighted_robinson_foulds_distance(familly['L_filter'], familly['R_filter'])
            
    
    parent_L = familly['R_filter']
    parent_R = familly['L_filter']
    for F in range(1,n_generations):
        
        def build_child(label_string, parent_, force= False):
            
            mutate = False
            
            familly[label_string] = parent_.extract_tree()

            if force or random.random() > 0.5:
                n = random.choice(list(familly[label_string].preorder_node_iter())[1:])
                familly[label_string].reroot_at_edge(n.edge,length1=0.5 * n.edge_length, 
length2=0.5 * n.edge_length)
                to_store[label_string + '_mutation']  = [x.taxon.label for x in n.leaf_nodes()]
                mutate = True
                

            else:
                to_store[label_string + '_mutation']  = None
                
            
            familly[label_string + '_filtered'] = familly[label_string].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
           
            return mutate
        
        
        rerooted = build_child('F{}L'.format(F),parent_L )
        if (rerooted):
            build_child('F{}R'.format(F), parent_R)
        else:
            build_child('F{}R'.format(F), parent_R, force = True)
            
        familly['F{}L_filtered'.format(F)].is_rooted = True
        familly['F{}R_filtered'.format(F)].is_rooted = True
        
        to_store['F{}_clade'.format(F)] = treecompare.symmetric_difference( familly['F{}L_filtered'.format(F)], familly['F{}R_filtered'.format(F)])

         
        familly['F{}L_filtered'.format(F)].is_rooted = False
        familly['F{}R_filtered'.format(F)].is_rooted = False
            
        to_store['F{}_URF'.format(F)] = treecompare.symmetric_difference(familly['F{}L_filtered'.format(F)], familly['F{}R_filtered'.format(F)])
        to_store['F{}_WRF'.format(F)] = treecompare.weighted_robinson_foulds_distance(familly['F{}L_filtered'.format(F)], familly['F{}R_filtered'.format(F)])
                
        parent_L = familly['F{}L'.format(F)]
        parent_R = familly['F{}R'.format(F)]
    
    
    

    forest.append(to_store)
        



In [36]:
import json

with open('distance_testing.json', 'w') as fp:
    json.dump(forest, fp)

In [633]:
n = 35
t1 = dendropy.Tree.get_from_string(forest[n]['L'], schema="newick", rooting="default-rooted")


print([ e.taxon for e in list(t1.leaf_node_iter()) ] )

print(forest[n]['L'])
print(forest[n]['R'])



print(forest[n]['L_filter'])







print(forest[n]['R_filter'])





[<Taxon 0x143c9bdc0 'T_4'>, <Taxon 0x11694fd30 'T_47'>, <Taxon 0x141e7c2e0 'T_36'>, <Taxon 0x116623b20 'T_6'>, <Taxon 0x115e6b130 'T_45'>, <Taxon 0x140d24520 'T_50'>, <Taxon 0x1444a7070 'T_17'>, <Taxon 0x1444a7250 'T_44'>, <Taxon 0x1444a7310 'T_26'>, <Taxon 0x1444a75b0 'T_13'>, <Taxon 0x1444a7670 'T_34'>, <Taxon 0x1444a7730 'T_37'>, <Taxon 0x1444a7850 'T_39'>, <Taxon 0x1444a7970 'T_21'>, <Taxon 0x1444a7a90 'T_46'>, <Taxon 0x1444a7b50 'T_28'>, <Taxon 0x1444a7c10 'T_16'>, <Taxon 0x1444a7d90 'T_27'>, <Taxon 0x1444a7e50 'T_11'>, <Taxon 0x1444a7f10 'T_35'>, <Taxon 0x1444821c0 'T_2'>, <Taxon 0x144482940 'T_48'>]
[&R] ((('T_4':0.06492961326792783,'T_47':0.06492961326792783):0.9614829204370432,(('T_36':0.16250371980926595,('T_6':0.06661375897490657,'T_45':0.06661375897490657):0.09588996083435938):0.6863426979933948,('T_50':0.5670205301446998,'T_17':0.5670205301446998):0.28182588765796085):1.0576218938454944):1.384148500889453,((('T_44':0.8988888623172768,'T_26':0.8988888623172768):0.7713429257

In [15]:
for i in forest:
    print(i['F1L_mutation'])

['T4']
['T2']
['T3']
None
['T_2']
['T_1']
['T1']
['T_6']
None
None
None
['T_14', 'T_3', 'T_10', 'T_6']
None
None
['T_9']
None
None
None
None
['T_15']
None
None
None
None
None
['T_32']
['T_2']
['T_17', 'T_36']
['T_13']
['T_19', 'T_32', 'T_37', 'T_17', 'T_20', 'T_2']
None
None
None
['T_11', 'T_40', 'T_9', 'T_42', 'T_20', 'T_3', 'T_17', 'T_16']
None
['T_8', 'T_23', 'T_22', 'T_32', 'T_41', 'T_45', 'T_42']
None
None
['T_31']
['T_24', 'T_17']
None
None
None
None
None
['T_64', 'T_52', 'T_34', 'T_4']
['T_19', 'T_67', 'T_35', 'T_18', 'T_57', 'T_46']
None
None
None
None
['T_3']
['T_19']
['T_10', 'T_39', 'T_71']
['T_20', 'T_73', 'T_79', 'T_36']
['T_64']
['T_23', 'T_14', 'T_28', 'T_53', 'T_49']
None
['T_43']
['T_46']
None
None
['T_11']
None
['T_27', 'T_51']
['T_62']
None
None
None
['T_20']
None
['T_32']
['T_35', 'T_67', 'T_49', 'T_6', 'T_22', 'T_77', 'T_7', 'T_105', 'T_4', 'T_83', 'T_39', 'T_2', 'T_18']
None
None
['T_19', 'T_66', 'T_101']
['T_7']
['T_25']
['T_5']
['T_88']
None
None
['T_116']
None


In [20]:
print(familly['F3R_filtered'].length())

150.8445231380462


In [26]:
print(familly['F3R_filtered'].length())
for e in familly['F3R_filtered'].postorder_edge_iter():
    
    if random.random() > 0.7:
        e.length = e.length *1.5
print(familly['F3R_filtered'].length()) 

166.72102239939835
190.48274808556755


In [52]:
tns = dendropy.TaxonNamespace(['T_{}'.format(i) for i in range(1,200)])
t1 = dendropy.Tree.get_from_string(forest[3]['R_filter'], schema="newick", rooting="default-rooted", taxon_namespace=tns )
t2 = dendropy.Tree.get_from_string(forest[3]['L_filter'], schema="newick", rooting="default-rooted", taxon_namespace=tns)

t1.is_rooted = False
t2.is_rooted = False

In [53]:
t1.print_plot(plot_metric='length')
t2.print_plot(plot_metric='length')

                                                       /------------------- T3 
                                                       |                       
                                                       |          /-- T1       
                                                /------+       /--+            
                                                |      |   /---+  \---- T_1    
                                                |      |   |   |               
                                                |      \---+   \--------- T2   
                                                +          |                   
                                                |          |         /T4       
                                                |          \---------+         
                                                |                    \T_3      
                                                |                              
                                        

In [54]:
treecompare.weighted_robinson_foulds_distance(t1, t2)
            

0.9898516274734948

In [60]:
t1.is_rooted = True
t2.is_rooted = True
t1.as_string(schema='newick')

"[&R] (T3:0.6593457333227907,(((T1:0.12592760869404268,'T_1':0.18889141304106402):0.09524798297977241,T2:0.33176338751072265):0.12118011121568753,(T4:0.0,'T_3':0.0):0.3423557028895026):0.14581217898853674,'T_2':0.8428976540823929):1.64374216955737;\n"

In [59]:
t2.as_string(schema='newick')

"[&R] (T3:0.4395638222151938,(((T1:0.12592760869404268,'T_1':0.12592760869404268):0.09524798297977241,T2:0.2211755916738151):0.12118011121568753,(T4:0.0,'T_3':0.0):0.3423557028895026):0.09720811932569116,'T_2':0.8428976540823929):1.0958281130382466;\n"

In [63]:
sum1 = 0
for e in t1.postorder_edge_iter():
    sum1 += e.length
print(sum1)

4.497163942281882


In [64]:
sum2 = 0
for e in t2.postorder_edge_iter():
    sum2 += e.length
print(sum2)

3.507312314808387


In [65]:
abs(sum1-sum2)

0.9898516274734952